## Image Classifier

The objective is to build an image classifier that is capable of properly identifying four different categories of image. 

The data consists of various train and test samples across the four categories of image. The data for a specific category is a singular image that has been flipped, rotated, or slightly altered in some way. 

### Preparation


In [22]:
#pip install --upgrade pip

In [23]:
#pip install tensorflow

In [14]:
from tensorflow import keras
import random


## 1. Data Processing
#### a) Use the "ImageDataGenerator()" class from keras.processing.image to build out an instance called "train_datagen" with the following parameters: 

rescale = 1./255

shear_range = 0.2

zoom_range = 0.2

horizontal_flip = True

In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

#### b) Then build training set by using the method ".flow_from_directory()"

In [16]:
train_set = train_datagen.flow_from_directory(
    'ass6_dataset_train', # replace with the path where your training data is stored
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 88 images belonging to 4 classes.


#### c) Observations:

The image shapes include arc, mailbox, cross, and complex structure. Images are all black and white. There are 4 classes. 


## 2. Initial Classifier Build: 

In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

classifier = Sequential()

classifier.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(64,64,3), activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())

classifier.add(Dense(units=128, activation='relu'))

classifier.add(Dense(units= 4 , # of classes
                     activation='softmax'))

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [18]:
classifier.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 12544)             0         
                                                                 
 dense_22 (Dense)            (None, 128)             

## 3. Model Runs: 

In [19]:
classifier.fit(train_set,
               steps_per_epoch=3,
               epochs=3)

Epoch 1/3
3/3 [==============================] - 1s 226ms/step - loss: 1.4101 - accuracy: 0.3182
Epoch 2/3
3/3 [==============================] - 1s 223ms/step - loss: 1.0660 - accuracy: 0.5682
Epoch 3/3
3/3 [==============================] - 1s 223ms/step - loss: 0.4886 - accuracy: 0.9318


In [20]:
classifier.save('classifier_ass6.h5')

In [21]:
import os, glob
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('classifier_ass6.h5')
print("Loaded model from disk")

# test data path
img_dir = "ass6_dataset_test" # Enter Directory of test set

# iterate over each test image
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)

# print the files in the dataset_test folder 
for f in files:
    print(f)
    
# make a prediction and add to results 
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results


Loaded model from disk
ass6_dataset_test/C033.png
ass6_dataset_test/1022.png
ass6_dataset_test/4011.png
ass6_dataset_test/1053.png
ass6_dataset_test/6051.png
ass6_dataset_test/4053.png
ass6_dataset_test/C014.png
ass6_dataset_test/6023.png
1/1 [==============================] - 0s 19ms/step


[array([3]),
 array([0]),
 array([0]),
 array([0]),
 array([1]),
 array([2]),
 array([1]),
 array([1])]

In [22]:
# check category labels in training_set
train_set.class_indices

{'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}

#### identify what category each images in test daset belongs to using images in the training set as references:

image 1022，1053 : 0

image 4011，4053: 2

image 6023, 6051: 1

image C014, C033: 3


In [23]:
test_label= [3, 0, 2, 0, 1, 2, 3, 1]

In [24]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_label, results)
print("Accuracy:", accuracy)

Accuracy: 0.75


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

results = []

# tuples with steps_per_epoch and epochs 
combinations = [(1,1), (1,2), (1,3), (2,4), (2,5), (2,6), (3,7), (3,8), (5,9), (5,10)]

test_label= [3, 0, 2, 0, 1, 2, 3, 1]

# loop over combinations and fit model
for step, epoch in combinations:
    # classifier
    classifier = Sequential()
    classifier.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(64,64,3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2,2)))
    classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2,2)))
    classifier.add(Flatten())
    classifier.add(Dense(units=128, activation='relu'))
    classifier.add(Dense(units=4, activation='softmax'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    #fix the warning ???
    for e in range(epoch):
        print(f'Epoch {e}')
        batches = 0
        for x, y in train_set:
          classifier.fit(x, y)
          batches += 1
          if batches >= step:
            break

    # fit classifier
    history = classifier.fit(train_set, steps_per_epoch=step, epochs=epoch, verbose=0)

    # get accuracy of model
    test_data = []
    
    img_dir = "ass6_dataset_test" # Enter Directory of test set
    test_data_path = os.path.join(img_dir, '*g')
    test_files = glob.glob(test_data_path)

    for f1 in test_files:
        img = image.load_img(f1, target_size=(64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        test_data.append(img)
    test_data = np.concatenate(test_data, axis=0)
    y_pred = classifier.predict(test_data)
    y_pred = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(test_label, y_pred)
   
    # append the results to the list of results
    results.append((step, epoch, accuracy))

# create a pandas DataFrame from the list of results
results_df = pd.DataFrame(results, columns=['Steps_per_epoch', 'Epochs', 'Accuracy'])
print(results_df)


Epoch 0
1/1 [==============================] - 0s 63ms/step
Epoch 0
1/1 [==============================] - 1s 581ms/step - loss: 1.4229 - accuracy: 0.2812
Epoch 1
1/1 [==============================] - 0s 55ms/step
Epoch 0
1/1 [==============================] - 0s 347ms/step - loss: 1.3998 - accuracy: 0.3125
Epoch 1
1/1 [==============================] - 0s 61ms/step - loss: 2.7987 - accuracy: 0.1250
Epoch 2
1/1 [==============================] - 0s 52ms/step
Epoch 0
1/1 [==============================] - 0s 62ms/step - loss: 1.2955 - accuracy: 0.1250
Epoch 1
1/1 [==============================] - 0s 57ms/step - loss: 1.0526 - accuracy: 0.6250
Epoch 2
1/1 [==============================] - 0s 53ms/step - loss: 0.7866 - accuracy: 0.7500
Epoch 3
1/1 [==============================] - 0s 55ms/step
Epoch 0
1/1 [==============================] - 0s 60ms/step - loss: 1.7442 - accuracy: 0.3125
Epoch 1
1/1 [==============================] - 0s 56ms/step - loss: 1.0353 - accuracy: 0.4583
Epoch 

In [25]:
# solution from isa. (warning as well...)

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

results = []
steps_ = []
epochs_ = []
accuracy_ = []

# tuples with steps_per_epoch and epochs 
combinations = [(1,1), (1,2), (1,3), (2,4), (2,5), (2,6), (3,7), (3,8), (5,9), (5,10)]

test_label= [3, 0, 2, 0, 1, 2, 3, 1]

# loop over combinations and fit model
for i in combinations:
    # classifier
    classifier = Sequential()
    classifier.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(64,64,3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2,2)))
    classifier.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2,2)))
    classifier.add(Flatten())
    classifier.add(Dense(units=128, activation='relu'))
    classifier.add(Dense(units=4, activation='softmax'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
            
    step, epoch = i
    steps_.append(step)
    epochs_.append(epoch)
    print(f"Training model with steps_per_epoch={step} and epochs={epoch}")
    
    #fix the warning ???
    for e in range(epoch):
        print(f'Epoch {e}')
        batches = 0
        for x, y in train_set:
          classifier.fit(x, y)
          batches += 1
          if batches >= step:
            break

    # fit classifier and save model
    classifier.fit(train_set, steps_per_epoch=step, epochs=epoch)
    classifier.save(f'model_{step}_{epoch}.h5')
    print("Saved model")

    # run model
    model = load_model(f'model_{step}_{epoch}.h5')
    print("Loaded model from disk")
    
    img_dir = "ass6_dataset_test" # Enter Directory of test set
    test_data_path = os.path.join(img_dir, '*g')
    test_files = glob.glob(test_data_path)
    
    test_data = []
    results = []
    for f1 in test_files:
        img = image.load_img(f1, target_size=(64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        test_data.append(img)
        result = model.predict(img)
        r = np.argmax(result, axis=1)
        results.append(r)
    
    #get accuracy 
    accuracy = accuracy_score(test_label, results)
    accuracy_.append(accuracy)
    print(f'accuracy of model: {accuracy}\n')
   

Training model with steps_per_epoch=1 and epochs=1
Epoch 0
1/1 [==============================] - 0s 464ms/step - loss: 2.5921 - accuracy: 0.2812
Saved model
Loaded model from disk
1/1 [==============================] - 0s 19ms/step
accuracy of model: 0.25

Training model with steps_per_epoch=1 and epochs=2
Epoch 0
1/1 [==============================] - 0s 414ms/step - loss: 1.4595 - accuracy: 0.1250
Epoch 1
1/1 [==============================] - 0s 256ms/step - loss: 2.1458 - accuracy: 0.4167
Epoch 1/2
1/1 [==============================] - 1s 527ms/step - loss: 1.0381 - accuracy: 0.5938
Epoch 2/2
1/1 [==============================] - 0s 316ms/step - loss: 1.0368 - accuracy: 0.4375
Saved model
Loaded model from disk
1/1 [==============================] - 0s 19ms/step
accuracy of model: 0.5

Training model with steps_per_epoch=1 and epochs=3
Epoch 0
1/1 [==============================] - 0s 345ms/step - loss: 1.3881 - accuracy: 0.2812
Epoch 1
1/1 [==============================] - 0s 

Epoch 2/6
2/2 [==============================] - 1s 293ms/step - loss: 0.3226 - accuracy: 0.9286
Epoch 3/6
2/2 [==============================] - 1s 208ms/step - loss: 0.1725 - accuracy: 0.9821
Epoch 4/6
2/2 [==============================] - 1s 201ms/step - loss: 0.1783 - accuracy: 0.9643
Epoch 5/6
2/2 [==============================] - 1s 290ms/step - loss: 0.1226 - accuracy: 0.9821
Epoch 6/6
2/2 [==============================] - 1s 204ms/step - loss: 0.1088 - accuracy: 0.9464
Saved model
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
accuracy of model: 0.875

Training model with steps_per_epoch=3 and epochs=7
Epoch 0
1/1 [==============================] - 0s 260ms/step - loss: 1.7561 - accuracy: 0.2500
Epoch 1
1/1 [==============================] - 0s 52ms/step - loss: 0.7275 - accuracy: 0.8333
Epoch 2
1/1 [==============================] - 0s 53ms/step - loss: 0.4474 - accuracy: 0.9167
Epoch 3
1/1 [==============================] - 0s 60ms/step - loss: 

1/1 [==============================] - 0s 58ms/step - loss: 0.5141 - accuracy: 0.9062
Epoch 2
1/1 [==============================] - 0s 53ms/step - loss: 0.1897 - accuracy: 0.9167
Epoch 3
1/1 [==============================] - 0s 61ms/step - loss: 0.2189 - accuracy: 0.9375
Epoch 4
1/1 [==============================] - 0s 57ms/step - loss: 0.1628 - accuracy: 0.9688
Epoch 5
1/1 [==============================] - 0s 55ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 6
1/1 [==============================] - 0s 65ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 7
1/1 [==============================] - 0s 59ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 8
1/1 [==============================] - 0s 61ms/step - loss: 0.0276 - accuracy: 1.0000
Epoch 1/9
5/5 [==============================] - 1s 143ms/step - loss: 0.0278 - accuracy: 1.0000
Saved model
Loaded model from disk
1/1 [==============================] - 0s 19ms/step
accuracy of model: 0.75

Training model with steps_per_epoch=5 and epoch

1/1 [==============================] - 0s 54ms/step - loss: 0.0262 - accuracy: 1.0000
Epoch 9
1/1 [==============================] - 0s 66ms/step - loss: 0.0343 - accuracy: 1.0000
Epoch 1/10
5/5 [==============================] - 1s 120ms/step - loss: 0.0210 - accuracy: 1.0000
Saved model
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
accuracy of model: 0.75



In [26]:
results_df = pd.DataFrame({'Steps_per_Epoch': steps_, 'Epochs': epochs_, 'Accuracy': accuracy_})
results_df

,Steps_per_Epoch,Epochs,Accuracy
0,1,1,0.250
1,1,2,0.500
2,1,3,0.750
3,2,4,0.750
4,2,5,0.750
5,2,6,0.875
6,3,7,0.750
7,3,8,0.750
8,5,9,0.750
9,5,10,0.750


### Discussions:

### 1. Effects of the following on accuracy and loss (train & test): 

#### Increasing the steps_per_epoch:

1) This will help improve accuracy as the model is exposed to more batches of data, which means more frequent updates on the weight and adjusting to the training data more effectively. It leads to better generalization of the model and better accuracy on both training and test data. Also, increasing the steps_per_epoch can help to reduce overfitting, as the model is being trained on more diverse data.

2) However, it can slow down training process, because model is being trained on more data. Also, it can increase the risk of overfitting, especially if the data is not diverse enough. Finally, it can increase the risk of the model overfitting to the training data if the number of epochs is not increased correspondingly. In such cases, the model may perform well on the training data, but poorly on the test data.

#### Increasing the number of epochs:

This usually leads to an improvement in accuracy, especially in the initial epochs. Because the network has more chances to learn the patterns in the training data, and to adjust the weights and biases to better fit the data. As the number of epochs increases, the loss on the training set generally decreases. But, the model may begin to overfit the training data, which can result in a decrease in accuracy on the test set, and an increase in the loss.


#### 2. Two uses of zero padding in CNN:

1) Keeping spatial dimensions of the input volume. This is important because convolutional layers can cause the spatial dimensions to shrink. By adding padding, we can control the output size of the convolutional layer and avoid losing spatial information.

2) Ensure that the convolution operation is performed over the entire input volume, including the boundary pixels. This is important because otherwise, the filter would not be able to process the pixels at the boundary of the input volume, resulting in a loss of information.


#### 3. use of a 1 x 1 kernel in CNN:

1) dimensionality reduction or feature transformation: It acts as a filter that performs a dot product between the input and the weight matrix, producing an output that is the sum of the weighted values. The 1x1 kernel is a convolutional filter with a width and height of 1, and it can be applied to a single channel or multiple channels in the input data.When a 1x1 convolution is applied to the output of a preceding layer in a CNN, it can change the number of filters (or channels) in the output. This is useful for reducing the number of feature maps in the output of a layer, and thus, reducing the computational cost of the CNN. It can also be used for mixing features from different channels, as the convolutional operation is performed across channels as well as spatial dimensions.

2) Another use is to create "bottleneck" layers in a CNN architecture, which improves computational efficiency and reduces # parameters required in the network. This approach involves using a 1x1 convolution to reduce the dimensionality of the input data before applying larger convolutions, which can reduce the number of computations required and make the network more efficient.


#### 4. Advantages of a CNN over a fully connected DNN for this image classification problem:

1) Higher Parameter Efficiency: CNN has smaller # parameters compared to fully connected DNN. In CNNs, the same filter is applied to different parts of the image, which makes the model more parameter efficient.

2) Local Connectivity: In CNN, each neuron is connected only to a small region of the input, which makes the network more efficient at processing high dimensional inputs like images.

3) Translation Invariance: CNNs can detect same features in different parts of the image. This is achieved by using filters that slide across the entire image. This helps to create a model that is invariant to translation in the input.

4) Regularization: CNN uses dropout and batch normalization techniques to prevent overfitting on the training data.

5) Hierarchical Learning: CNN can learn hierarchical representations of the input. The first layer of the CNN learns simple patterns like edges, lines and corners, and the subsequent layers learn more complex features like curves, shapes and textures. This makes the CNN better at recognizing complex objects.

